# Assessment Title: Cycling 

Analyzing Ireland's Cycling Data and Comparing It with Mexico's and New York's

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import ListedColormap
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
img = mpimg.imread('bike.jpg')
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

Main libraries

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

# DUBLIN 

Import of raw data and data cleaning 

In [ ]:
dublin1= pd.read_csv('dublin1.csv')
dublin2= pd.read_csv('dublin2.csv')

In [ ]:
dublin1.head(1)

In [ ]:
dublin2.head(1)

In [ ]:
# headers of each file
headers_dublin1 = dublin1.columns
headers_dublin2 = dublin2.columns

# Comparing the headers
if list(headers_dublin1) == list(headers_dublin2):
    print("The datasets have the same headers.")
else:
    print("The datasets have different headers.")

Since they have the same headers they will be concatenated to have only one dataset 

In [ ]:
dublin = pd.concat([dublin1, dublin2])
dublin.head(3)

### Exploratory Data Analysis (EDA)

In [ ]:
dublin.shape

In [ ]:
dublin.info()

In [ ]:
dublin.isnull().sum()

In [ ]:
dublin['system_id'].value_counts()

In [ ]:
dublin['is_installed'].value_counts()

In [ ]:
dublin['is_renting'].value_counts()

In [ ]:
dublin['is_returning'].value_counts()

In [ ]:
dublin= dublin.drop(['short_name', 'region_id','system_id','is_installed','is_renting','is_returning'], axis=1)

In [ ]:
dublin.describe()

#### Date and time format change 

In [ ]:
dublin['last_reported'] = pd.to_datetime(dublin['last_reported'])

In [ ]:
# Creating new columns 'date' and 'time' 
dublin['date'] = dublin['last_reported'].dt.date
dublin['time'] = dublin['last_reported'].dt.time

In [ ]:
dublin= dublin.drop('last_reported', axis=1)

#### Period of time 

In [ ]:
min_date = dublin['date'].min()
max_date = dublin['date'].max()

print("The 'date' column ranges from {} to {}.".format(min_date, max_date))

In [ ]:
dublin.head(1)

In [ ]:
dublin= dublin.drop('address', axis=1)

## Analysis of bicycle use in Dublin

### Bikes available

In [ ]:
print("Descriptive statistics for number of bikes available:")
print(dublin['num_bikes_available'].describe())

In [ ]:
sns.boxplot(x=dublin['num_bikes_available'], color='skyblue')
plt.title('Number of Bikes Available')
plt.xlabel('Number of Bikes Available')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

The mean number of bicycles available per docks is 11 bicycles per station. 

### Stations

In [ ]:
stations = dublin['station_id'].nunique()
print(f"Total number of stations: {stations}")

In [ ]:
#Stations with Highest bike availability
top_stations = dublin.groupby('name')['num_bikes_available'].mean().sort_values(ascending=False)
print("Stations with highest bike availability:")
print(top_stations.head(10))

In [ ]:
# Stations with lowest bike availability
print("Stations with lowest bike availability:")
print(top_stations.tail(10))

In [ ]:
top_10_stations = top_stations.head(10)

plt.figure(figsize=(9, 6))
top_10_stations.plot(kind='bar', color='pink')


plt.title('Top 10 Stations by Average Number of Bikes Available',fontsize=13)
plt.xlabel('Station')
plt.ylabel('Average Number of Bikes Available')
plt.xticks(rotation=20, fontsize=6) 


plt.tight_layout()  
plt.show()

## Location of stations with dataset information

### Scatter plot

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly"

In [ ]:
fig = px.scatter(
    dublin,
    x='lon',
    y='lat',
    size='capacity',
    color='capacity',
    hover_name='name',
    hover_data={
        'station_id': True,
        'num_bikes_available': True,
        'num_docks_available': True,
        'date': True,
        'time': True
    },
    size_max=60,
    title='Bike Station Capacity',
    labels={'capacity': 'Bike Station Capacity'}
)

# layout 
fig.update_layout(
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    title_font_size=14,
    xaxis_title_font_size=12,
    yaxis_title_font_size=12,
    coloraxis_colorbar_title='Capacity'
)

fig.show()

## Location of stations using an API 

In [ ]:
!pip install requests

In [ ]:
import requests
import json

In [ ]:
api_key = '5c20182a11586d9a8e24446b2d69e8c33ff0948c'
contract_name = 'Irelandbikes'

In [ ]:
# Defining the API URL
url = f'https://api.jcdecaux.com/vls/v1/stations?contract={contract_name}&apiKey={api_key}'

In [ ]:
response = requests.get(url)

Since JCDecaux has different contracts in various locations around the world, to obtain accurate information, the contract locations must first be checked to access it.

In [ ]:
# API's Key
api_key = '5c20182a11586d9a8e24446b2d69e8c33ff0948c'

# Defining URL 
contracts_url = f'https://api.jcdecaux.com/vls/v1/contracts?apiKey={api_key}'

contracts_response = requests.get(contracts_url)

# Checking if the request was successful
if contracts_response.status_code == 200:
    contracts_data = contracts_response.json()
    # Contracts
    for contract in contracts_data:
        print(contract['name'])
else:
    print(f'Error: {contracts_response.status_code}')

In [ ]:
# Defining API and contract's name
api_key = '5c20182a11586d9a8e24446b2d69e8c33ff0948c'
contract_name = 'Dublin'

url = 'https://api.jcdecaux.com/vls/v1/stations'

# Making the request to the API
params = {
    'contract': contract_name,
    'apiKey': api_key
}

response = requests.get(url, params=params)

# Verification    
data = response.json()
if response.status_code == 200:
        print('The request was successful.')
else:
    print('The request was not successful.')

In [ ]:
dublin2 = pd.DataFrame(data)
dublin2.head(2)

In [ ]:
pip install folium

In [ ]:
import folium

def create_station_map(dublin2):
    # Create a map centered on Dublin
    dublin_map = folium.Map(location=[53.3498, -6.2603], zoom_start=13)
    
    # Iterate over the rows of the DataFrame
    for index, row in dublin2.iterrows():
        # Get the position 
        lat = row['position']['lat']
        lng = row['position']['lng']
        station_name = row['name']
        
        # Add a marker to the map
        folium.Marker([lat, lng], popup=station_name).add_to(dublin_map)
    
    # Display the map
    display(dublin_map)

create_station_map(dublin2)

# NEW YORK

Import of raw data and data cleaning 

In [ ]:
ny = pd.read_csv('NY1.csv')

In [ ]:
dublin.head(1)

In [ ]:
ny.head(3)

In [ ]:
ny.info()

In [ ]:
ny.isnull().sum()

In [ ]:
ny= ny.drop(['ride_id','start_station_id', 'end_station_id'], axis=1)

#### Null field

start_station_name 

In [ ]:
empty_name = ny[ny['start_station_name'].isnull()]
empty_name

In [ ]:
lat = ny[ny['start_lat'] == 40.71]
lat

In [ ]:
ny['start_station_name'] = ny['start_station_name'].fillna('other')

#### Date and time format change

In [ ]:
ny['started_at'] = pd.to_datetime(ny['started_at'], format='%d/%m/%Y %H:%M')
ny['ended_at'] = pd.to_datetime(ny['ended_at'], format='%d/%m/%Y %H:%M')

In [ ]:
# Extracting date and time
ny['date_started'] = ny['started_at'].dt.date
ny['date_end'] = ny['ended_at'].dt.date
ny['time_started'] = ny['started_at'].dt.time
ny['time_end'] = ny['ended_at'].dt.time

In [ ]:
ny.head(2)

In [ ]:
ny['date_started'] = ny['date_started'].astype(str)
ny['time_started'] = ny['time_started'].astype(str)
ny['date_end'] = ny['date_end'].astype(str)
ny['time_end'] = ny['time_end'].astype(str)

In [ ]:
ny['duration_minutes'] = (ny['ended_at'] - ny['started_at']).astype('timedelta64[m]')

#### Duration of each ride

In [ ]:
ny['duration'] = ny['ended_at'] - ny['started_at']

# In minutes
ny['duration_minutes'] = ny['duration'].dt.total_seconds() / 60
ny['duration_minutes'] = ny['duration_minutes'].round(2)

ny = ny.drop('duration', axis=1)

In [ ]:
ny= ny.drop(['started_at','ended_at'], axis=1)

In [ ]:
ny.head(1)

In [ ]:
ny.isnull().sum()

In [ ]:
# NY = Individual analysis of New York
# ny = Analysis comparing New York with other countries 

NY= ny

In [ ]:
NY= NY.dropna(subset=['end_lat', 'end_lng','end_station_name'])
NY.isnull().sum()

In [ ]:
ny= ny.drop(['rideable_type','end_lat','end_lng','member_casual','end_station_name'], axis=1)
ny.isnull().sum()

In [ ]:
ny.dtypes

In [ ]:
ny.describe()

In [ ]:
ny.shape

# Analysis of bicycle use in New York

#### Type of bicycle

In [ ]:
print("Bicycle Type available:")
print(NY['rideable_type'].value_counts())

In [ ]:
rideable_type_counts = NY['rideable_type'].value_counts()
rideable_type_counts.plot(kind='bar', color='darkseagreen')
plt.title('Bicycle Type', fontsize=14)
plt.xlabel('Type' , fontsize=13)
plt.ylabel('Bicycles', fontsize=13)
plt.xticks(rotation=360)
plt.show()

#### Membership Type

In [ ]:
print("Rides by Membership Type:")
print(NY['member_casual'].value_counts())

In [ ]:
membership_counts = NY['member_casual'].value_counts()

plt.figure(figsize=(5, 6))
plt.pie(membership_counts, labels=membership_counts.index, autopct='%1.1f%%', colors=['khaki', 'darkseagreen'], wedgeprops=dict(width=0.3))
plt.title('Rides by Membership Type')
plt.show()

### Duration of Rides

In [ ]:
print("Duration of Rides:")
print(NY['duration_minutes'].describe())

##### Long Trips

In [ ]:
long_trips = ny[ny['duration_minutes'] > 20]

num_long_trips = long_trips.shape[0]

print(f"Number of rides longer than 20 minutes: {num_long_trips}")

In [ ]:
long_trips = (num_long_trips / ny.shape[0]) * 100
print(f'Percentage of trips more than 20 minutes: {long_trips:.2f}%')

##### Short Trips

In [ ]:
short_trips = ny[ny['duration_minutes'] < 20]

num_short_trips = short_trips.shape[0]

print(f'Number of trips less than 20 minutes: {num_short_trips}')

In [ ]:
short_trips = (num_short_trips / ny.shape[0]) * 100
print(f'Percentage of trips less than 20 minutes: {short_trips:.2f}%')

In [ ]:
labels = ['Trips < 20 min', 'Trips >= 20 min']
sizes = [num_short_trips, num_long_trips]
colors = ['darkseagreen', 'khaki']
explode = (0.1, 0) 

plt.figure(figsize=(6, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=140)
plt.title('Proportion of Trip Durations')
plt.show()

In [ ]:
#Distribution of Ride Durations Up to 60 minutes
plt.figure(figsize=(10, 6))
plt.hist(ny['duration_minutes'], bins=range(0, 60, 1), color='khaki', edgecolor='black')
plt.title('Distribution of Ride Durations, Up to 60 minutes')
plt.xlabel('Minutes)')
plt.ylabel('Frequency')
plt.xlim(0, 60)
plt.show()

## Data normalization 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
durations = ny[['duration_minutes']]

scaler = MinMaxScaler()

ny['duration_minutes_normalized'] = scaler.fit_transform(durations)

ny[['duration_minutes', 'duration_minutes_normalized']].head()

## Data Standardized

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

ny['duration_minutes_standardized'] = scaler.fit_transform(durations)


ny[['duration_minutes', 'duration_minutes_standardized']].head()

## Range

In [ ]:
bins = [0, 5, 10, 15, 20, 30, 60, 120, ny['duration_minutes'].max()]
labels = ['0-5', '5-10', '10-15', '15-20', '20-30', '30-60', '60-120', '120+']
ny['duration_range'] = pd.cut(ny['duration_minutes'], bins=bins, labels=labels, right=False)

In [ ]:
duration_range_counts = ny['duration_range'].value_counts(normalize=True) * 100

In [ ]:
plt.figure(figsize=(10, 6))
ax = duration_range_counts.sort_index().plot(kind='bar', color='lightseagreen', edgecolor='black')
for i in ax.patches:
    ax.text(i.get_x() + i.get_width() / 2, i.get_height() + 0.5, f'{i.get_height():.1f}%', ha='center', va='bottom')

plt.title('Percentage Distribution of Trip Duration')
plt.xlabel('Duration Range (minutes)')
plt.ylabel('Percentage')
plt.xticks(rotation=360)
plt.show()

### Heatmaps for Bike Trip Start and End Locations

In [ ]:
from folium.plugins import HeatMap
from IPython.display import IFrame

In [ ]:
# Sample of the DataFrame for a more manageable size
sample_df = NY.sample(frac=0.01, random_state=1) 

# Map 
avg_lat = sample_df['start_lat'].mean()
avg_lng = sample_df['start_lng'].mean()
station_map = folium.Map(location=[avg_lat, avg_lng], zoom_start=13)

# Coordinates for the heatmap
start_coords = sample_df[['start_lat', 'start_lng']].dropna().values.tolist() 
end_coords = sample_df[['end_lat', 'end_lng']].dropna().values.tolist() 

# Heatmap to the map
HeatMap(start_coords, radius=10, max_zoom=13).add_to(station_map)
HeatMap(end_coords, radius=10, max_zoom=13).add_to(station_map)

station_map.save('heatmap.html')

In [ ]:
IFrame(src='heatmap.html', width=800, height=600)

#### TOP 10 Routes

In [ ]:
NY['route'] = NY['start_station_name'] + ' to ' + NY['end_station_name']
route_usage = NY['route'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=route_usage.values, y=route_usage.index, palette='viridis')
plt.title('Top 10 Most Common Routes')
plt.xlabel('Number of Trips')
plt.ylabel('Route')
plt.show()

# MEXICO CITY

In [ ]:
mexico = pd.read_csv('mexico.csv')
mexico.head()

#### Exploratory Data Analysis (EDA)

In [ ]:
mexico.columns.tolist()

In [ ]:
mexico_columns = [
    'ride_gender',
    'ride_age',
    'Bike',
    'start_station_name',
    'date_started',
    'time_started',
    'end_station_name',
    'date_end',
    'time_end'
]

mexico.columns = mexico_columns

In [ ]:
mexico.dtypes

In [ ]:
mexico.shape

In [ ]:
mexico.info()

In [ ]:
mexico.isnull().sum()

In [ ]:
# mexico = Analysis comparing Mexico City with other countries 
# mexico1= Individual analysis of Mexico City

mexico1= mexico

In [ ]:
mexico= mexico.drop(['ride_age','Bike','end_station_name'], axis=1)
mexico.isnull().sum()

In [ ]:
mexico1= mexico1.dropna(subset=['ride_age'])
mexico1.isnull().sum()

# Analysis of bicycle use in Mexico City

#### Gender

In [ ]:
gender_counts = mexico1['ride_gender'].value_counts()
print(f"Registered genders:\n{gender_counts}")

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x=gender_counts.index, y=gender_counts.values, palette='crest')
plt.title('Distribution of Rides by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Rides')
plt.show()

#### AGE

In [ ]:
print("Cyclists age:")
print(mexico1['ride_age'].describe())

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(mexico1['ride_age'], bins=range(int(mexico1['ride_age'].min()), int(mexico1['ride_age'].max()) + 1, 1), edgecolor='black')
plt.title('Distribution of Cyclists\' Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

#### Rides by Bike ID

In [ ]:
top_bikes = mexico1['Bike'].value_counts().head(10)

top_bikes_data = mexico1[mexico1['Bike'].isin(top_bikes.index)]

print("Top 10 most used bikes and their details:")
print(top_bikes_data[['ride_gender', 'ride_age', 'Bike', 'start_station_name']])

In [ ]:
plt.barh(top_bikes.index.astype(str), top_bikes.values, color='skyblue', edgecolor='black')
plt.title('Top 10 Most Used Bikes')
plt.xlabel('Rides')
plt.ylabel('Bike ID')
plt.grid(axis='x', linestyle='--', alpha=0.7)

In [ ]:
sns.boxplot(x='Bike', y='ride_age', data=top_bikes_data, order=top_bikes.index)
plt.title('Age Distribution for Top 10 Most Used Bikes')
plt.xlabel('Bike ID')
plt.ylabel('Age')
plt.xticks(rotation=20)
plt.show()